In [1]:
import multiprocessing as mp
import time
from urllib.request import urlopen, urljoin
from bs4 import BeautifulSoup
import re

base_url = "https://morvanzhou.github.io/"

# crawl用来爬取网页

In [2]:
def crawl(url):
    #使用urlopen打开网页
    response = urlopen(url)
    #返回原始的HTML页面
    return response.read().decode()

# parse用来解析网页，在HTML页面中找到有用的信息

In [3]:
def parse(html):
    soup = BeautifulSoup(html, 'lxml')
    urls = soup.find_all('a',{"href": re.compile('^/.+?/$')})
    title = soup.find('h1').get_text().strip()
    page_urls = set([urljoin(base_url, url['href']) for url in urls])
    url = soup.find('meta', {'property': "og:url"})['content']
    return title, page_urls, url

# 单线程

In [4]:
unseen = set([base_url,])#使用set功能，用来区分爬取和未爬取的网页
seen = set()

count, t1 = 1, time.time()


if base_url != "http://127.0.0.1:4000/":
    restricted_crawl = True
else:
    restricted_crawl = False
    
    
while len(unseen) != 0:
    if restricted_crawl and len(seen) > 20:
        break
        
    print('\nDistribute Crawling...')
    htmls = [crawl(url) for url in unseen]
    
    print('\nDistribute Parsing...')
    results = [parse(html) for html in htmls]
    
    print('\nAnalysing...')
    seen.update(unseen)
    unseen.clear()
    
    for title, page_urls, url in results:
        print(count, title, url)
        count += 1
        unseen.update(page_urls - seen)
print('Total time: %.lf s' % (time.time() - t1,))


Distribute Crawling...

Distribute Parsing...

Analysing...
1 教程 https://morvanzhou.github.io/

Distribute Crawling...

Distribute Parsing...

Analysing...
2 计算机视觉 教程系列 https://morvanzhou.github.io/tutorials/machine-learning/computer-vision/
3 关于莫烦 https://morvanzhou.github.io/about/
4 机器学习实践 https://morvanzhou.github.io/tutorials/machine-learning/ML-practice/
5 有趣的机器学习系列 https://morvanzhou.github.io/tutorials/machine-learning/ML-intro/
6 Theano 教程系列 https://morvanzhou.github.io/tutorials/machine-learning/theano/
7 基础教程系列 https://morvanzhou.github.io/tutorials/python-basic/basic/
8 其他教学系列 https://morvanzhou.github.io/tutorials/others/
9 说吧~ https://morvanzhou.github.io/discuss/
10 高级爬虫: 让 Selenium 控制你的浏览器帮你爬 https://morvanzhou.github.io/tutorials/data-manipulation/scraping/5-01-selenium/
11 Tensorflow 教程系列 https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/
12 Git 版本管理 教程系列 https://morvanzhou.github.io/tutorials/others/git/
13 Threading 多线程教程系列 https://morvanzhou.githu

# 多线程

In [5]:
unseen = set([base_url,])
seen = set()

pool = mp.Pool(4)  #创建进程池
count, t1 = 1, time.time()
while len(unseen) != 0:
    if restricted_crawl and len(seen) > 20:
        break
        
    print('\nDistribute Crawling...')
    crawl_jobs = [pool.apply_async(crawl, args=(url,)) for url in unseen]
    htmls = [j.get() for j in crawl_jobs]
    
    print('\nDistribute Parsing...')
    parse_jobs = [pool.apply_async(parse, args=(html,)) for html in htmls]
    results = [j.get() for j in parse_jobs]
    
    print('\nAnalysing...')
    seen.update(unseen)
    unseen.clear()
    
    for title, page_urls, url in results:
        print(count, title, url)
        count += 1
        unseen.update(page_urls - seen)
print('Total time: %.lf s' % (time.time() - t1,))


Distribute Crawling...

Distribute Parsing...

Analysing...
1 教程 https://morvanzhou.github.io/

Distribute Crawling...

Distribute Parsing...

Analysing...
2 计算机视觉 教程系列 https://morvanzhou.github.io/tutorials/machine-learning/computer-vision/
3 关于莫烦 https://morvanzhou.github.io/about/
4 机器学习实践 https://morvanzhou.github.io/tutorials/machine-learning/ML-practice/
5 有趣的机器学习系列 https://morvanzhou.github.io/tutorials/machine-learning/ML-intro/
6 Theano 教程系列 https://morvanzhou.github.io/tutorials/machine-learning/theano/
7 基础教程系列 https://morvanzhou.github.io/tutorials/python-basic/basic/
8 其他教学系列 https://morvanzhou.github.io/tutorials/others/
9 说吧~ https://morvanzhou.github.io/discuss/
10 Git 版本管理 教程系列 https://morvanzhou.github.io/tutorials/others/git/
11 高级爬虫: 让 Selenium 控制你的浏览器帮你爬 https://morvanzhou.github.io/tutorials/data-manipulation/scraping/5-01-selenium/
12 Tensorflow 教程系列 https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/
13 Threading 多线程教程系列 https://morvanzhou.githu